In [266]:
import re
import time
import requests
import selenium
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from sympy import symbols, solve
from plotly.subplots import make_subplots
from pandas_datareader import data as pdr
from selenium.webdriver.common.by import By
from forex_python.converter import CurrencyRates
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService

In [267]:
ticker = str(input('Ticker: '))

In [268]:
options = Options()

options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

financial_url = fr'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
driver.get(financial_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

## In case SA does not have the Tax Rate data

# pretax_income_parent = soup.find('div', class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Pretax Income")
# pretax_income_span = pretax_income_parent.find_next('span').find_next('span')
# pretax_income_value = int(re.sub(',', '', pretax_income_span.text)) * 1000

# tax_provision_parent = soup.find('div',  class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Tax Provision")
# tax_provision_span = tax_provision_parent.find_next('span').find_next('span')
# tax_provision_value = int(re.sub(',', '', tax_provision_span.text)) * 1000

interest_expense_parent = soup.find('div',  class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Interest Expense")
interest_expense_span = interest_expense_parent.find_next('span').find_next('span')
interest_expense_value = int(re.sub(',', '', interest_expense_span.text)) * 1000

driver.quit()

In [269]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

treasury_bond_url = r'https://ycharts.com/indicators/10_year_treasury_rate'
driver.get(treasury_bond_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

tnx_row = soup.find('td', string = 'Last Value').find_next('td').text
tnx_value = float(re.sub('%', '', tnx_row))

driver.quit()

In [270]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

financial_url = r'https://finance.yahoo.com/quote/VTI/performance?p=VTI'
driver.get(financial_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

return_div = soup.find('span', string = '10-Year').find_parent('div')
vti_10_year_return = return_div.find_all('span', class_ = 'W(20%)')[0].text.strip()
market_value = float(re.sub('%', '', vti_10_year_return))

print(f'Expected Market Return (10-year VTI): {market_value} %')
print(f'The value of 10-year Treasury Bond Yield is: {tnx_value} %')
print(f'Market Premium: {round(market_value - tnx_value, 4)} %')

driver.quit()

Expected Market Return (10-year VTI): 11.91 %
The value of 10-year Treasury Bond Yield is: 4.26 %
Market Premium: 7.65 %


In [294]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

share_url = fr'https://stockanalysis.com/stocks/{ticker}/statistics/'
driver.get(share_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

## Market Cap
href = f"/stocks/{ticker.lower()}/market-cap/"
market_cap_td = soup.find('a', class_ = "dothref text-default", attrs = {'href': href}).find_parent('td').find_next('td')['title']
market_cap_num = int(re.sub(',', '', market_cap_td))

## Beta
beta_td = soup.find('span', string = "Beta (1Y) ").find_parent('td').find_next('td')['title']
beta_num = float(beta_td)

## Revenue
href = f"/stocks/{ticker.lower()}/revenue/"
rev_td = soup.find('a', attrs = {'href': href}, class_ = "dothref text-default").find_parent('td').find_next('td')['title']
rev_num = int(re.sub(',', '', rev_td))

## Free Cash Flow
fcf_td = soup.find('span', string = 'Free Cash Flow ').find_parent('td').find_next('td')['title']
fcf_num = int(re.sub(',', '', fcf_td))

## Tax Rate
tax_td = soup.find('span', string = 'Effective Tax Rate ').find_parent('td').find_next('td').text
tax_num = float(re.sub('%', '', tax_td))

## Revenue Growth (5-Year)
rev_growth_td = soup.find('span', string = 'Revenue Growth Forecast (5Y) ').find_parent('td').find_next('td').text
rev_growth_num = float(re.sub('%', '', rev_growth_td)) / 100 + 1

## EV / EBIT multiple
ebit_td = soup.find('span', string = 'EBIT ').find_parent('td').find_next('td')['title']
ebit_num = float(re.sub(',', '', ebit_td))

evebit_td = soup.find('span', string = 'EV / EBIT ').find_parent('td').find_next('td').text
evebit_num = float(evebit_td)

## EV / EBITDA multiple
ebitda_td = soup.find('span', string = 'EBITDA ').find_parent('td').find_next('td')['title']
ebitda_num = float(re.sub(',', '', ebitda_td))

evebitda_td = soup.find('span', string = 'EV / EBITDA ').find_parent('td').find_next('td').text
evebitda_num = float(evebitda_td)

## Cash & Cash Equivalents
ccs_td = soup.find('span', string = 'Cash & Cash Equivalents ').find_parent('td').find_next('td')['title']
ccs_num = float(re.sub(',', '', ccs_td))

## Total Debt
debt_td = soup.find('span', string = 'Total Debt ').find_parent('td').find_next('td')['title']
debt_num = float(re.sub(',', '', debt_td))

## Share Outstandings
share_td = soup.find('span', string = 'Shares Outstanding ').find_parent('td').find_next('td')['title']
share_num = float(re.sub(',', '', share_td))

## Current Stock Price
## During 休盤
price = float(re.sub(',', '', soup.find('div', class_ = "text-4xl font-bold block sm:inline").text))

## During 開盤
# price = float(soup.find('div', class_ = "text-4xl font-bold inline-block").text)

driver.quit()

$WACC\ (Weighted\ Average\ Cost\ of\ Capital) = W_D * R_D * (1-t) + W_E * R_E$ <br><br>
Weight of Debt ($W_D$) = $\Large\frac{Debt}{Debt\ +\ Market\ Cap}$ <br><br>
Weight of Equity ($W_E$) = $\Large\frac{Market\ Cap}{Debt\ +\ Market\ Cap}$ <br><br>
Cost of Debt ($R_D$) = $\Large\frac{Interest\ Expense}{Debt}$ <br><br>
Cost of Equity ($R_E$) = $Treasury\ Bond\ Rate\ +\ Beta * (Expected\ Market\ Return - Treasury\ Bond\ Rate\ )$

In [276]:
weight_of_debt = debt_num / (debt_num + market_cap_num)
weight_of_equity = market_cap_num / (debt_num + market_cap_num)
cost_of_debt = interest_expense_value/ debt_num
cost_of_equity = (tnx_value + beta_num * (market_value - tnx_value)) / 100
wacc = (weight_of_debt * cost_of_debt * (1 - tax_num / 100) + weight_of_equity * cost_of_equity) * 100

print(f'Weight of Debt = {round(weight_of_debt * 100, 4)} %')
print(f'Weight of Equity = {round(weight_of_equity * 100, 4)} %')
print(f'Cost of Debt = {round(cost_of_debt * 100, 4)} %')
print(f'Cost of Euqity = {round(cost_of_equity * 100, 4)} %')
print('========================')
print(f'WACC = {round(wacc, 4)} %')

Weight of Debt = 6.0715 %
Weight of Equity = 93.9285 %
Cost of Debt = 4.1347 %
Cost of Euqity = 13.9296 %
WACC = 13.318 %


In [290]:
# options = Options()
# options.add_argument('--headless')
# options.add_argument('window-size = 800x600')
# prefs = {"profile.managed_default_content_settings.images": 2}
# options.add_experimental_option("prefs", prefs)

# driver = webdriver.Chrome(options = options)

# sa_url = fr'https://stockanalysis.com/stocks/{ticker}/forecast/'
# driver.get(sa_url)
# page_source = driver.page_source
# soup = BeautifulSoup(page_source, 'html.parser')

# revenue_growth_numbers = []

# revenue_growth_td = soup.find('tr', class_ = "border-b text-[0.85rem] dark:border-dark-700 sm:text-[0.95rem]").find_all('td')

# for td in range(6, 11):
#     revenue_growth_numbers.append(float(re.sub('%', '', revenue_growth_td[td]['title'])))

# growth_rate_1 = (revenue_growth_numbers[0] / 100) + 1
# growth_rate_2 = (revenue_growth_numbers[1] / 100) + 1
# growth_rate_3 = (revenue_growth_numbers[2] / 100) + 1
# growth_rate_4 = (revenue_growth_numbers[3] / 100) + 1
# growth_rate_5 = (revenue_growth_numbers[4] / 100) + 1
terminal_growth_rate = 1.025
# print(f"Growth Rate of Further 5 years and Terminal Growth Rate: {revenue_growth_numbers[0]}%, {revenue_growth_numbers[1]}%, {revenue_growth_numbers[2]}%, {revenue_growth_numbers[3]}%, {revenue_growth_numbers[4]}%, {round((terminal_growth_rate - 1) * 100, 3)}%")

# driver.quit()

### Terminal Value
[Wall Street Oasis | Terminal Value](https://www.wallstreetoasis.com/resources/skills/valuation/terminal-value)  
[Wall Street Oasis | EBIT vs EBITDA](https://www.wallstreetoasis.com/resources/skills/finance/exit-multiple)  

#### Exit Multiple Method
The exit multiple method assumes that a company will **be sold after the forecast period** for a multiple of some market indicator.  
$$
Terminal\ Value\ = Final\ Year\ Metric\ *\ Exit\ Multiple
$$

Advantages:  
- Reflecting the market expectations and valuation multiples of comparable companies.
- Using market-based data rather than subjective assumptions about growth rates and margins.

Drawbacks:
- It is more difficult and subjective to find and select appropriate multiples and comparable companies.
- It is more uncertain and volatile, depending on market conditions and sentiments that may change over time or vary across different sectors.

##### EBITDA multiple
- **Mature & Stable** companies with consistent cash flows and margins
- It may NOT be suitable for **high-growth** or **low-margin** companies like technology or biotech.

##### EBIT multiple
- Include depreciation and amortization, which is particularly useful while analyzing **capital-intensive** businesses where depreciation is a true economic cost.
- Such as, automobile manufacturing, energy, steel production, and telecommunications.

##### Revenue multiple
- Often used for **high-growth** or **low-margin** companies with strong revenue potential but **not profitable** or have **negative cash flows**.
- It may NOT be suitable for companies with **stable revenue but declining growth**.

##### Earnings multiple
- Often used for profitable and growing companies with **positive cash flows and earnings**, such as **consumer discretionary**.

---

#### Perpetual Growth Method
The perpetual growth method assumes that a company will **always produce cash flows at a steady rate** after the projection time.  
$$
Terminal\ Value = \frac{Final\ Year\ FCF\ *\ (1\ +\ Perpetuity\ Growth\ Rate)}{(Discount\ Rate\ -\ Perpetuity\ Growth\ Rate)}
$$

Advantages:
- Consist with the theory of discounted cash flow valuation.
- It is widely used and accepted by academics and practitioners.

Drawbacks:
- It is sensitive to the assumptions of the growth rate and the discount rate.
- Improbable in a dynamic and competitive market.
- This is because it ignores the potential changes in the industry structure.

[Enterprise Value Multiples by Sector (US)](https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/vebitda.html)

In [291]:
f = '/Users/shenchingfeng/Stock-Valuation/EV_EBITDA by Industry.csv'
indus_ebitda_ratio = pd.read_csv(f)
indus_ebitda_ratio.dropna(how = 'any', inplace = True)

row_index = indus_ebitda_ratio[indus_ebitda_ratio['Exchange:Ticker'].str.contains(ticker)].index

if not row_index.empty:

    industry_group = indus_ebitda_ratio.loc[row_index, 'Industry Group'].values[0]
    print(f"The Industry Group for {ticker} is: {industry_group}")
    
else:

    print(f"No matching row found for {ticker}")
    
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

indus_ebitda_url = 'https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/vebitda.html'
driver.get(indus_ebitda_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

indus_ebit_td = soup.find('td', string = industry_group).find_next('td').find_next('td').find_next('td').find_next('td').text
indus_ebit_num = float(indus_ebit_td)

indus_ebitda_td = soup.find('td', string = industry_group).find_next('td').find_next('td').find_next('td').text
indus_ebitda_num = float(indus_ebitda_td)

The Industry Group for AVGO is: Semiconductor


In [295]:
current_datetime = datetime.now()
current_year = current_datetime.year
end_of_year = datetime(current_datetime.year, 12, 31)
time_difference = end_of_year - current_datetime
percentage_remaining = (time_difference.days / 365)

fcf_ttm = round(fcf_num)
# fcf_1 = round(fcf_ttm * growth_rate_1)
fcf_1 = round(fcf_ttm * rev_growth_num)
dcf_1 = round(fcf_1 / ((1 + (wacc / 100)) ** (percentage_remaining / 2)))  ## DCF in the middle of this year whatever the remaining

# fcf_2 = round(fcf_1 * growth_rate_2)
fcf_2 = round(fcf_1 * rev_growth_num)
dcf_2 = round(fcf_2 / ((1 + (wacc / 100)) ** (percentage_remaining + 0.5))) ## The end of this year + 0.5 year = Middle of the next year

# fcf_3 = round(fcf_2 * growth_rate_3)
fcf_3 = round(fcf_2 * rev_growth_num)
dcf_3 = round(fcf_3 / ((1 + (wacc / 100)) ** (percentage_remaining + 1.5)))

# fcf_4 = round(fcf_3 * growth_rate_4)
fcf_4 = round(fcf_3 * rev_growth_num)
dcf_4 = round(fcf_4 / ((1 + (wacc / 100)) ** (percentage_remaining + 2.5)))

# fcf_5 = round(fcf_4 * growth_rate_5)
fcf_5 = round(fcf_4 * rev_growth_num)
dcf_5 = round(fcf_5 / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

print(f"TTM Revenue: {rev_num:,}")
print("==================================================================")

print(f"EBIT: {round(ebit_num):,}")
print(f"EBIT margin: {round(ebit_num * 100 / rev_num, 2)} %")
print(f"Average Industrial EV / EBIT of {industry_group}: {indus_ebit_num}")
print(f"EV / EBIT of {ticker}: {evebit_num:,}")
print(f"Terminal Value by EBIT multiple: {round(ebit_num * evebit_num):,}")

print("==================================================================")
print(f"EBITDA: {round(ebitda_num):,}")
print(f"EBITDA margin: {round(ebitda_num * 100 / rev_num, 2)} %")
print(f"Average Industrial EV / EBITDA of {industry_group}: {indus_ebitda_num}")
print(f"EV / EBITDA of {ticker}: {evebitda_num:,}")
print(f"Terminal Value by EBITDA multiple: {round(ebitda_num * evebitda_num):,}")

print("==================================================================")
print(f"Terminal Value by Perpetual Growth Method: {round(fcf_5 * terminal_growth_rate / ((wacc / 100) - (terminal_growth_rate - 1))):,}")

## Perpetual Growth Method
# dcf_terminal = round((fcf_5 * terminal_growth_rate / ((wacc / 100) - (terminal_growth_rate - 1))) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

## Exit Multiple Method EV/EBIT, Capital-Intensive Industries
# dcf_terminal = round((ebit_num * evebit_num) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

## Exit Multiple Method EV/EBITDA, Mature & Stable Industries
dcf_terminal = round((ebitda_num * evebitda_num) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

ev = dcf_1 + dcf_2 + dcf_3 + dcf_4 + dcf_5 + dcf_terminal

print("==================================================================")
print(f"Latest {current_year - 1} Free Cash Flow:          {fcf_num:,}")
print(f"Free Cash Flow margin: {round(fcf_num * 100 / rev_num, 2)} %")
print(f"")
print(f"Predicted {current_year} Discounted Cash Flow: {dcf_1:,}")
print(f"Predicted {current_year + 1} Discounted Cash Flow: {dcf_2:,}")
print(f"Predicted {current_year + 2} Discounted Cash Flow: {dcf_3:,}")
print(f"Predicted {current_year + 3} Discounted Cash Flow: {dcf_4:,}")
print(f"Predicted {current_year + 4} Discounted Cash Flow: {dcf_5:,}")
print(f"Predicted Terminal value of Discounted Cash Flow:  {dcf_terminal:,}")
print("==================================================================")
print(f"Net Present Value of {ticker}: {ev:,}")

TTM Revenue: 35,819,000,000
EBIT: 16,719,000,000
EBIT margin: 46.68 %
Average Industrial EV / EBIT of Semiconductor: 53.32
EV / EBIT of AVGO: 37.8
Terminal Value by EBIT multiple: 631,978,200,000
EBITDA: 20,554,000,000
EBITDA margin: 57.38 %
Average Industrial EV / EBITDA of Semiconductor: 31.59
EV / EBITDA of AVGO: 30.74
Terminal Value by EBITDA multiple: 631,829,960,000
Terminal Value by Perpetual Growth Method: 339,856,060,388
Latest 2023 Free Cash Flow:          17,633,000,000
Free Cash Flow margin: 49.23 %

Predicted 2024 Discounted Cash Flow: 19,276,168,806
Predicted 2025 Discounted Cash Flow: 19,795,465,950
Predicted 2026 Discounted Cash Flow: 20,134,707,875
Predicted 2027 Discounted Cash Flow: 20,479,763,509
Predicted 2028 Discounted Cash Flow: 20,830,732,484
Predicted Terminal value of Discounted Cash Flow:  366,931,902,969
Net Present Value of AVGO: 467,448,741,593


In [297]:
equity = ev - debt_num + ccs_num
target_price = round(equity / share_num, 2)

print(f"Target Price: {target_price}")
print(f"Current Price: {price}")
print(f"Margin of Safety: {round(((target_price - price) / price) * 100, 2)} %")

Target Price: 945.03
Current Price: 1296.37
Margin of Safety: -27.1 %


In [298]:
g = (price * share_num - dcf_terminal + debt_num - ccs_num) / fcf_num
# c = terminal_growth_rate / (((wacc / 100) - (terminal_growth_rate - 1)) * ((1 + wacc / 100) ** 5))

x = symbols('x')
quintic_equation = x**5 + x**4 + x**3 + x**2 + x - g
solutions = solve(quintic_equation, x)
numerical_solutions = [sol.evalf() for sol in solutions]
t = float(numerical_solutions[0])
r = t * (1 + wacc / 100) - 1

print(f'Implied 5-year Growth Rate by current price: {round(r * 100, 4)} %')

Implied 5-year Growth Rate by current price: 57.7978 %
